In [1]:
from google.colab import drive
# drive.mount('/content/drive/MyDrive/IITP/sohyun/creditcard_prediction/data')
drive.mount('/content/drive')

%cd drive/MyDrive/IITP/sohyun/creditcard_prediction/data
!pwd

Mounted at /content/drive
/content/drive/MyDrive/IITP/sohyun/creditcard_prediction/data
/content/drive/MyDrive/IITP/sohyun/creditcard_prediction/data


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import time
import datetime
from torch.autograd import grad
from torch.autograd import Variable
# from model import *
import matplotlib.pyplot as plt
# from utils import *
# from data_loader import *
import IPython
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.preprocessing import StandardScaler
import itertools
# from utils import *

class EarlyStopping:
  def __init__(self, patience=30):
      self.loss = np.inf
      self.patience = 0
      self.patience_limit = patience
      
  def step(self, loss):
      if self.loss > loss:
          self.loss = loss
          self.patience = 0
      else:
          self.patience += 1
  
  def is_stop(self):
      return self.patience >= self.patience_limit

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df, eval_mode):
        self.df = df
        self.eval_mode = eval_mode
        if self.eval_mode:
            self.labels = self.df['Class'].values
            self.df = self.df.drop(columns=['Class']).values
        else:
            self.df = self.df.values
        
    def __getitem__(self, index):
        if self.eval_mode :
            self.x = self.df[index]
            self.y = self.labels[index]
            return torch.Tensor(self.x), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x), 0
        
    def __len__(self):
        return len(self.df)

# train_dataset = MyDataset(df=train_df, eval_mode=False)
# train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True)

# val_dataset = MyDataset(df = val_df, eval_mode=True)
# val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False)


"""Build and return data loader."""
train_df = pd.read_csv('./train.csv')
train_df = train_df.drop(columns=['ID'])
val_df = pd.read_csv('./val.csv')
val_df = val_df.drop(columns=['ID'])
test_df = pd.read_csv('./test.csv')
test_df = test_df.drop(columns=['ID'])

#-------------------#
#---# Normalize #---#
#-------------------#
# case 1 - standardscaler
scaler_n = StandardScaler()
scaler_n.fit(train_df.values)

val_x = val_df.drop(columns=['Class'])
train_x_scaleN = pd.DataFrame(scaler_n.transform(train_df.values), columns = train_df.columns) # 확인 : train_x_scaleN.mean(), train_x_scaleN.var()
val_x_scaleN = pd.DataFrame(scaler_n.transform(val_x.values), columns = val_x.columns)
test_x_scaleN = pd.DataFrame(scaler_n.transform(test_df.values), columns = test_df.columns)

train_df = train_x_scaleN
val_df = pd.concat([val_x_scaleN, pd.DataFrame(val_df['Class'])])
test_df = test_x_scaleN

# dataset = MyDataset(data_path, mode)
train_dataset = MyDataset(df=train_df, eval_mode=False)
valid_dataset = MyDataset(df=val_df, eval_mode=True)
test_dataset = MyDataset(df=test_df, eval_mode=False)

shuffle = False
if mode == 'train': shuffle = True

data_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=shuffle)
data_loader_v = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=shuffle)
data_loader_test = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=shuffle)
return data_loader, data_loader_v, data_loader_test